# Analyzing the generated simulation parameters #

## A NOTE BEFORE STARTING ##

Since the ``prismatique`` git repository tracks this notebook under its original
basename ``analyzing_sim_params.ipynb``, we recommend that you copy the original
notebook and rename it to any other basename that is not one of the original
basenames that appear in the ``<root>/examples`` directory before executing any
of the notebook cells below, where ``<root>`` is the root of the ``prismatique``
repository. This way you can explore the notebook by executing and modifying
cells without changing the original notebook, which is being tracked by git.

## Table of contents ##

- [Import necessary modules](#Import-necessary-modules)
- [Introduction](#Introduction)
- [Set paths to files storing simulation parameter sets](#Set-paths-to-files-storing-simulation-parameter-sets)
- [Load simulation parameter sets and represent them as Python objects](#Load-simulation-parameter-sets-and-represent-them-as-Python-objects)
- [Extracting properties that are well-defined for all three simulation parameter sets](#Extracting-properties-that-are-well-defined-for-all-three-simulation-parameter-sets)
  - [Extract the beam defocii](#Extract-the-beam-defocii)
  - [Extract the sample unit cell dimensions in units of $\mathring{\mathrm{A}}$](#Extract-the-sample-unit-cell-dimensions-in-units-of-$\mathring{\mathrm{A}}$)
  - [Extract the sample supercell dimensions in units of $\mathring{\mathrm{A}}$](#Extract-the-sample-supercell-dimensions-in-units-of-$\mathring{\mathrm{A}}$)
  - [Extract the lateral dimensions of the sample supercell in units of pixels](#Extract-the-lateral-dimensions-of-the-sample-supercell-in-units-of-pixels)
  - [Extract the lateral pixel size of the sample supercell in units of $\mathring{\mathrm{A}}$](#Extract-the-lateral-pixel-size-of-the-sample-supercell-in-units-of-$\mathring{\mathrm{A}}$)
  - [Extract the slice thickness of the sample supercell in units of $\mathring{\mathrm{A}}$](#Extract-the-slice-thickness-of-the-sample-supercell-in-units-of-$\mathring{\mathrm{A}}$)
  - [Extract the number of slices used to partition the sample](#Extract-the-number-of-slices-used-to-partition-the-sample)
  - [Extract the number of frozen phonon configuration subsets](#Extract-the-number-of-frozen-phonon-configuration-subsets)
  - [Extract the number of frozen phonon configurations in each subset](#Extract-the-number-of-frozen-phonon-configurations-in-each-subset)
  - [Extract the total number of frozen phonon configurations](#Extract-the-total-number-of-frozen-phonon-configurations)
  - [Extract the data size of each subset of potential slices that one could generate according the sample model](#Extract-the-data-size-of-each-subset-of-potential-slices-that-one-could-generate-according-the-sample-model)
  - [Extract the data size of a set of potential slices that one could generate according the sample model](#Extract-the-data-size-of-a-set-of-potential-slices-that-one-could-generate-according-the-sample-model)
- [Extracting properties that are only well-defined for both STEM simulation parameter sets](#Extracting-properties-that-are-only-well-defined-for-both-STEM-simulation-parameter-sets)
  - [Extract the probe scan pattern type](#Extract-the-probe-scan-pattern-type)
  - [Extract the underlying grid dimensions of the probe scan pattern](#Extract-the-underlying-grid-dimensions-of-the-probe-scan-pattern)
  - [Extract the probe positions](#Extract-the-probe-positions)
  - [Extract the output layer depths](#Extract-the-output-layer-depths)
  - [Extract the data size of the STEM simulation output datasets that one could generate according to the STEM simulation parameter sets](#Extract-the-data-size-of-the-STEM-simulation-output-datasets-that-one-could-generate-according-to-the-STEM-simulation-parameter-sets)
- [Extracting properties that are only well-defined for the STEM simulation parameter set that specifies that the PRISM algorithm be used](#Extracting-properties-that-are-only-well-defined-for-the-STEM-simulation-parameter-set-that-specifies-that-the-PRISM-algorithm-be-used)
  - [Extract the $\mathbf{k}_{xy}$-momentum vectors of the plane waves to be used to generate the $S$-matrices of the STEM simulation](#Extract-the-kxy-momentum-vectors-of-the-plane-waves-to-be-used-to-generate-the-S-matrices-of-the-STEM-simulation)
  - [Extract the data size of each subset of $S$-matrices that one could generate according the sample model](#Extract-the-data-size-of-each-subset-of-S-matrices-that-one-could-generate-according-the-sample-model)
  - [Extract the data size of a set of $S$-matrices that one could generate according the sample model](#Extract-the-data-size-of-a-set-of-S-matrices-that-one-could-generate-according-the-sample-model)
- [Extracting properties that are only well-defined for the HRTEM simulation parameter set](#Extracting-properties-that-are-only-well-defined-for-the-HRTEM-simulation-parameter-set)
  - [Extract the beam tilt step size](#Extract-the-beam-tilt-step-size)
  - [Extract the beam tilts](#Extract-the-beam-tilts)
  - [Extract the data size of the HRTEM simulation output dataset that one could generate according to the HRTEM simulation parameter set](#Extract-the-data-size-of-the-HRTEM-simulation-output-dataset-that-one-could-generate-according-to-the-HRTEM-simulation-parameter-set)

## Import necessary modules ##

In [1]:
# For general array handling.
import numpy as np



# The library that is the subject of this demonstration. Users may receive a
# warning message about ``pyopencl`` upon importing ``prismatique``. This
# message can be safely ignored.
import prismatique

## Introduction ##

In this notebook, we demonstrate how one can analyze the simulation parameter
sets that are generated by the script
``<root>/examples/sim_param_generator/generate.py``. Said script generates
serialized representations of three simulation parameter sets: one for
simulating a HRTEM experiment; and the other two for simulating a STEM
experiment, where each simulation parameter set specifies a different algorithm
to be used. For the current notebook to work properly, one must first generate
the simulation parameters by running the script.

In order to execute the cells in this notebook as intended, a set of Python
libraries need to be installed in the Python environment within which the cells
of the notebook are to be executed. For this particular notebook, users need to
install:

    pyprismatic>=2.0
    jupyter
    prismatique

The easiest way to install these libraries is within a `conda` virtual
environment. It is recommended that `pyprismatic` be installed separately,
prior to installing any of the other Python libraries listed above. GPU
acceleration is available for `pyprismatic` (and thus `prismatique`) if the
following conditions are met:

1. You are using a Linux or Windows machine that has NVIDIA GPUs.
2. A NVIDIA driver is installed with CUDA version 10.2.89 or greater.

If the above conditions have been met, and you would like to be able to use GPUs
with `prismatique`, run the following command:

    conda install -c conda-forge pyprismatic=2.*=gpu* cudatoolkit==<X>.<Y>.*

where `<X>` and `<Y>` are the major and minor versions of CUDA installed on
your machine, e.g. CUDA version 10.2.89 has a major version of `10`, and a
minor version of `2`. Users can omit `cudatoolkit==<X>.<Y>.*` if they do not
require a specific version of `cudatoolkit`, which should apply to most
scenarios. For CPU support only, run the following command instead:

    conda install -c conda-forge pyprismatic=2.*=cpu*

Once `pyprismatic` is installed, you can install the remaining libraries via
`pip` by running the following command:

    pip install prismatique[examples]

or alternatively, via `conda` by running the following command:

    conda install -c conda-forge prismatique jupyter

You can find the documentation for the `prismatique` library
[here](https://mrfitzpa.github.io/prismatique/_autosummary/prismatique.html). It
is recommended that you consult the documentation of this library as you explore
the notebook. Moreover, users should execute the cells in the order that they
appear, i.e. from top to bottom, as some cells reference variables that are set
in other cells above them. **Users should make sure to navigate the
documentation for the version of `prismatique` that they are currently using.**

## Set paths to files storing simulation parameter sets ##

In [2]:
# Path to file storing the serialized representation of the parameter set for
# the STEM simulation implemented using the multislice algorithm.
path_to_multislice_stem_sim_params = \
    "../data/sim_param_generator_output/multislice_stem_sim_params.json"

# Path to file storing the serialized representation of the parameter set for
# the STEM simulation implemented using the PRISM algorithm.
path_to_prism_stem_sim_params = \
    "../data/sim_param_generator_output/prism_stem_sim_params.json"

# Path to file storing the serialized representation of the parameter set for
# the HRTEM simulation.
path_to_hrtem_sim_params = \
    "../data/sim_param_generator_output/hrtem_sim_params.json"

## Load simulation parameter sets and represent them as Python objects ##

In [3]:
# Load serialized representation of the parameter set for the STEM simulation
# implemented using the multislice algorithm and represent it as a Python
# object.
multislice_stem_sim_params = \
    prismatique.stem.sim.Params.load(path_to_multislice_stem_sim_params)

# Load serialized representation of the parameter set for the STEM simulation
# implemented using the PRISM algorithm and represent it as a Python object.
prism_stem_sim_params = \
    prismatique.stem.sim.Params.load(path_to_prism_stem_sim_params)

# Load serialized representation of the parameter set for the HRTEM simulation
# and represent it as a Python object.
hrtem_sim_params = \
    prismatique.hrtem.sim.Params.load(path_to_hrtem_sim_params)

## Extracting properties that are well-defined for all three simulation parameter sets ##

### Extract the sample model parameters and represent it as a Python object ###

Each simulation parameter set specifies the same sample model. We can extract
the sample model parameters from any of the three simulation parameter sets
above.

In [4]:
hrtem_system_model_params = \
    hrtem_sim_params.core_attrs["hrtem_system_model_params"]
sample_specification = \
    hrtem_system_model_params.core_attrs["sample_specification"]

We make use of the object ``sample_specification`` further below to extract certain properties of the sample.

### Extract the beam defocii ###

Load the beam defocii specified in each simulation parameter set. Upon running a
simulation according to a given simulation parameter set, ``prismatique`` would
simulate a series of temporally coherent HRTEM or STEM experiments, where each
simulated experiment would use a different beam defocus from the set of defocii
specified in said simulation parameter set.

In [5]:
# From the parameter set for the STEM simulation implemented using the
# multislice algorithm.
defocii = prismatique.load.defocii(path_to_multislice_stem_sim_params)

unformatted_msg = "defocii (in Å): {}"
msg = unformatted_msg.format(defocii)
print(msg)

defocii (in Å): (-8156.921938165306, -3999.9999999999995, 156.92193816530607)


In [6]:
# From the parameter set for the STEM simulation implemented using the PRISM
# algorithm.
defocii = prismatique.load.defocii(path_to_prism_stem_sim_params)

unformatted_msg = "defocii (in Å): {}"
msg = unformatted_msg.format(defocii)
print(msg)

defocii (in Å): (-8156.921938165306, -3999.9999999999995, 156.92193816530607)


In [7]:
# From the parameter set for the HRTEM simulation.
defocii = prismatique.load.defocii(path_to_hrtem_sim_params)

unformatted_msg = "defocii (in Å): {}"
msg = unformatted_msg.format(defocii)
print(msg)

defocii (in Å): (-8156.921938165306, -3999.9999999999995, 156.92193816530607)


### Extract the sample unit cell dimensions in units of $\mathring{\mathrm{A}}$ ###

In [8]:
sample_unit_cell_dims = prismatique.sample.unit_cell_dims(sample_specification)

unformatted_msg = "sample unit cell dimensions (in Å): {}"
msg = unformatted_msg.format(sample_unit_cell_dims)
print(msg)

sample unit cell dimensions (in Å): (22.1228, 21.89589348896272, 15.54092)


### Extract the sample supercell dimensions in units of $\mathring{\mathrm{A}}$ ###

In [9]:
sample_supercell_dims = prismatique.sample.supercell_dims(sample_specification)

unformatted_msg = "sample supercell dimensions (in Å): {}"
msg = unformatted_msg.format(sample_supercell_dims)
print(msg)

sample supercell dimensions (in Å): (22.1228, 21.89589348896272, 15.54092)


### Extract the lateral dimensions of the sample supercell in units of pixels ###

In [10]:
sample_supercell_xy_dims_in_pixels = \
    prismatique.sample.supercell_xy_dims_in_pixels(sample_specification)

unformatted_msg = "lateral dimensions of sample supercell (in pixels): {}"
msg = unformatted_msg.format(sample_supercell_xy_dims_in_pixels)
print(msg)

lateral dimensions of sample supercell (in pixels): (128, 128)


### Extract the lateral pixel size of the sample supercell in units of $\mathring{\mathrm{A}}$ ###

In [11]:
sample_supercell_lateral_pixel_size = \
    prismatique.sample.supercell_lateral_pixel_size(sample_specification)

unformatted_msg = "sample supercell lateral pixel size (in Å): {}"
msg = unformatted_msg.format(sample_supercell_lateral_pixel_size)
print(msg)

sample supercell lateral pixel size (in Å): (0.172834375, 0.17106166788252125)


### Extract the slice thickness of the sample supercell in units of $\mathring{\mathrm{A}}$ ###

In [12]:
sample_supercell_slice_thickness = \
    prismatique.sample.supercell_slice_thickness(sample_specification)

unformatted_msg = "sample supercell slice thickness (in Å): {}"
msg = unformatted_msg.format(sample_supercell_slice_thickness)
print(msg)

sample supercell slice thickness (in Å): 0.9713085


### Extract the number of slices used to partition the sample ###

In [13]:
num_slices = prismatique.sample.num_slices(sample_specification)

unformatted_msg = "# of sample slices: {}"
msg = unformatted_msg.format(num_slices)
print(msg)

# of sample slices: 16


### Extract the number of frozen phonon configuration subsets ###

In [14]:
num_frozen_phonon_config_subsets = \
    prismatique.sample.num_frozen_phonon_config_subsets(sample_specification)

unformatted_msg = "# frozen phonon configuration subsets: {}"
msg = unformatted_msg.format(num_frozen_phonon_config_subsets)
print(msg)

# frozen phonon configuration subsets: 2


### Extract the number of frozen phonon configurations in each subset ###

In [15]:
func_alias = prismatique.sample.num_frozen_phonon_configs_in_subset

for subset_idx in range(num_frozen_phonon_config_subsets):
    num_frozen_phonon_configs_in_subset = func_alias(sample_specification,
                                                     subset_idx)
    
    unformatted_msg = "# frozen phonon configurations in subset {}: {}"
    msg = unformatted_msg.format(subset_idx, 
                                 num_frozen_phonon_configs_in_subset)
    print(msg)

# frozen phonon configurations in subset 0: 5
# frozen phonon configurations in subset 1: 5


### Extract the total number of frozen phonon configurations ###

In [16]:
total_num_frozen_phonon_configs = \
    prismatique.sample.total_num_frozen_phonon_configs(sample_specification)

unformatted_msg = "total # of frozen phonon configurations: {}"
msg = unformatted_msg.format(total_num_frozen_phonon_configs)
print(msg)

total # of frozen phonon configurations: 10


### Extract the data size of each subset of potential slices that one could generate according the sample model ###

In [17]:
func_alias = prismatique.sample.potential_slice_subset_data_size

for subset_idx in range(num_frozen_phonon_config_subsets):
    potential_slice_subset_data_size = func_alias(sample_specification,
                                                  subset_idx)
    
    unformatted_msg = "data size of potential slice subset {} (in bytes): {}"
    msg = unformatted_msg.format(subset_idx, 
                                 potential_slice_subset_data_size)
    print(msg)

data size of potential slice subset 0 (in bytes): 5242880
data size of potential slice subset 1 (in bytes): 5242880


### Extract the data size of a set of potential slices that one could generate according the sample model ###

In [18]:
potential_slice_set_data_size = \
    prismatique.sample.potential_slice_set_data_size(sample_specification)

unformatted_msg = "data size of potential slice set (in bytes): {}"
msg = unformatted_msg.format(potential_slice_set_data_size)
print(msg)

data size of potential slice set (in bytes): 10485760


## Extracting properties that are only well-defined for both STEM simulation parameter sets ##

### Extract the probe scan pattern type ###

This can be done in two ways. The first method involves the use of the function
``prismatique.load.scan_pattern_type``:

In [19]:
# From the the parameter set for the STEM simulation implemented using the
# multislice algorithm.
scan_pattern_type = \
    prismatique.load.scan_pattern_type(path_to_multislice_stem_sim_params)

unformatted_msg = "scan pattern type: {}"
msg = unformatted_msg.format(scan_pattern_type)
print(msg)

scan pattern type: jittered rectangular grid


In [20]:
# From the the parameter set for the STEM simulation implemented using the PRISM
# algorithm.
scan_pattern_type = \
    prismatique.load.scan_pattern_type(path_to_prism_stem_sim_params)

unformatted_msg = "scan pattern type: {}"
msg = unformatted_msg.format(scan_pattern_type)
print(msg)

scan pattern type: jittered rectangular grid


The second method involves the use of the function
``prismatique.scan.pattern_type``, and the STEM system model parameters. Both
STEM simulation parameter sets share the same STEM system model parameters,
hence can extract the STEM system model parameters from either STEM simulation
parameter set:

In [21]:
stem_system_model_params = \
    multislice_stem_sim_params.core_attrs["stem_system_model_params"]
scan_config = \
    stem_system_model_params.core_attrs["scan_config"]
scan_pattern_type = \
    prismatique.scan.pattern_type(scan_config)

unformatted_msg = "scan pattern type: {}"
msg = unformatted_msg.format(scan_pattern_type)
print(msg)

scan pattern type: jittered rectangular grid


### Extract the underlying grid dimensions of the probe scan pattern ###

This can be done in two ways. The first method involves the use of the function
``prismatique.load.grid_dims_in_units_of_probe_shifts``:

In [22]:
# From the the parameter set for the STEM simulation implemented using the
# multislice algorithm.
func_alias = \
    prismatique.load.grid_dims_in_units_of_probe_shifts
grid_dims_in_units_of_probe_shifts = \
    func_alias(path_to_multislice_stem_sim_params)

unformatted_msg = "probe scan grid dimensions (in units of probe shifts): {}"
msg = unformatted_msg.format(grid_dims_in_units_of_probe_shifts)
print(msg)

probe scan grid dimensions (in units of probe shifts): (3, 3)


In [23]:
# From the the parameter set for the STEM simulation implemented using the PRISM
# algorithm.
func_alias = \
    prismatique.load.grid_dims_in_units_of_probe_shifts
grid_dims_in_units_of_probe_shifts = \
    func_alias(path_to_prism_stem_sim_params)

unformatted_msg = "probe scan grid dimensions (in units of probe shifts): {}"
msg = unformatted_msg.format(grid_dims_in_units_of_probe_shifts)
print(msg)

probe scan grid dimensions (in units of probe shifts): (3, 3)


The second method involves the use of the function
``prismatique.scan.grid_dims_in_units_of_probe_shifts``:

In [24]:
func_alias = \
    prismatique.scan.grid_dims_in_units_of_probe_shifts
grid_dims_in_units_of_probe_shifts = \
    func_alias(sample_specification, scan_config)

unformatted_msg = "probe scan grid dimensions (in units of probe shifts): {}"
msg = unformatted_msg.format(grid_dims_in_units_of_probe_shifts)
print(msg)

probe scan grid dimensions (in units of probe shifts): (3, 3)


### Extract the probe positions ###

This can be done in two ways. The first method involves the use of the function
``prismatique.load.probe_positions``:

In [25]:
# From the the parameter set for the STEM simulation implemented using the
# multislice algorithm.
kwargs = {"filename": path_to_multislice_stem_sim_params,
          "force_2_col_shape": True}
probe_positions = prismatique.load.probe_positions(**kwargs)

msg = "probe positions (in Å):"
print(msg)
print(np.array(probe_positions))

probe positions (in Å):
[[ 4.85785417  5.29325906]
 [ 5.0139072  10.94690794]
 [ 5.61514736 15.35137548]
 [11.37311278  5.09270143]
 [10.25119624  9.43681506]
 [ 9.81603198 15.93636896]
 [16.49432862  5.37968288]
 [15.62138678  9.27052936]
 [15.87140464 16.31118136]]


In [26]:
# From the the parameter set for the STEM simulation implemented using the PRISM
# algorithm.
kwargs = {"filename": path_to_prism_stem_sim_params,
          "force_2_col_shape": True}
probe_positions = prismatique.load.probe_positions(**kwargs)

msg = "probe positions (in Å):"
print(msg)
print(np.array(probe_positions))

probe positions (in Å):
[[ 4.85785417  5.29325906]
 [ 5.0139072  10.94690794]
 [ 5.61514736 15.35137548]
 [11.37311278  5.09270143]
 [10.25119624  9.43681506]
 [ 9.81603198 15.93636896]
 [16.49432862  5.37968288]
 [15.62138678  9.27052936]
 [15.87140464 16.31118136]]


The second method involves the use of the function
``prismatique.scan.generate_probe_positions``:

In [27]:
kwargs = {"sample_specification": sample_specification,
          "scan_config": scan_config,
          "filename": None}
probe_positions = prismatique.scan.generate_probe_positions(**kwargs)

msg = "probe positions (in Å):"
print(msg)
print(np.array(probe_positions))

probe positions (in Å):
[[ 4.85785417  5.29325906]
 [ 5.0139072  10.94690794]
 [ 5.61514736 15.35137548]
 [11.37311278  5.09270143]
 [10.25119624  9.43681506]
 [ 9.81603198 15.93636896]
 [16.49432862  5.37968288]
 [15.62138678  9.27052936]
 [15.87140464 16.31118136]]


### Extract the output layer depths ###

This can be done in two ways. The first method involves the use of the function
``prismatique.load.output_layer_depths``:

In [28]:
# From the the parameter set for the STEM simulation implemented using the
# multislice algorithm.
output_layer_depths = \
    prismatique.load.output_layer_depths(path_to_multislice_stem_sim_params)

unformatted_msg = "output layer depths (in Å): {}"
msg = unformatted_msg.format(output_layer_depths)
print(msg)

output layer depths (in Å): (3.885234, 7.770468, 11.655702, 15.54092)


In [29]:
# From the the parameter set for the STEM simulation implemented using the
# PRISM algorithm.
output_layer_depths = \
    prismatique.load.output_layer_depths(path_to_prism_stem_sim_params)

unformatted_msg = "output layer depths (in Å): {}"
msg = unformatted_msg.format(output_layer_depths)
print(msg)

output layer depths (in Å): (15.54092,)


The second method involves the use of the function
``prismatique.stem.output.layer_depths``:

In [30]:
# From the the parameter set for the STEM simulation implemented using the
# multislice algorithm.
output_params = multislice_stem_sim_params.core_attrs["output_params"]
alg_specific_output_params = output_params.core_attrs["alg_specific_params"]

kwargs = {"sample_specification": sample_specification,
          "alg_specific_params": alg_specific_output_params}
output_layer_depths = prismatique.stem.output.layer_depths(**kwargs)

unformatted_msg = "output layer depths (in Å): {}"
msg = unformatted_msg.format(output_layer_depths)
print(msg)

output layer depths (in Å): (3.885234, 7.770468, 11.655702, 15.54092)


In [31]:
# From the the parameter set for the STEM simulation implemented using the PRISM
# algorithm.
output_params = prism_stem_sim_params.core_attrs["output_params"]
alg_specific_output_params = output_params.core_attrs["alg_specific_params"]

kwargs = {"sample_specification": sample_specification,
          "alg_specific_params": alg_specific_output_params}
output_layer_depths = prismatique.stem.output.layer_depths(**kwargs)

unformatted_msg = "output layer depths (in Å): {}"
msg = unformatted_msg.format(output_layer_depths)
print(msg)

output layer depths (in Å): (15.54092,)


### Extract the data size of the STEM simulation output datasets that one could generate according to the STEM simulation parameter sets ###

In [32]:
# From the the parameter set for the STEM simulation implemented using the
# multislice algorithm.
output_params = multislice_stem_sim_params.core_attrs["output_params"]
kwargs = {"stem_system_model_params": stem_system_model_params,
          "output_params": output_params}
stem_output_data_size = prismatique.stem.output.data_size(**kwargs)

unformatted_msg = "data size of STEM simulation output (in bytes): {}"
msg = unformatted_msg.format(stem_output_data_size)
print(msg)

data size of STEM simulation output (in bytes): 35442864


In [33]:
# From the the parameter set for the STEM simulation implemented using the PRISM
# algorithm.
output_params = prism_stem_sim_params.core_attrs["output_params"]
kwargs = {"stem_system_model_params": stem_system_model_params,
          "output_params": output_params}
stem_output_data_size = prismatique.stem.output.data_size(**kwargs)

unformatted_msg = "data size of STEM simulation output (in bytes): {}"
msg = unformatted_msg.format(stem_output_data_size)
print(msg)

data size of STEM simulation output (in bytes): 8860716


## Extracting properties that are only well-defined for the STEM simulation parameter set that specifies that the PRISM algorithm be used ##

### Extract the $\mathbf{k}_{xy}$-momentum vectors of the plane waves to be used to generate the $S$-matrices of the STEM simulation ###

This can be done in two ways. The first method involves the use of the function
``prismatique.load.S_matrix_k_xy_vectors``:

In [34]:
S_matrix_k_xy_vectors = \
    prismatique.load.S_matrix_k_xy_vectors(path_to_prism_stem_sim_params)

msg = "S-matrix k_xy vectors (in 1/Å):"
print(msg)
print(np.array(S_matrix_k_xy_vectors))

S-matrix k_xy vectors (in 1/Å):
[[ 0.          0.        ]
 [ 0.          0.04567066]
 [ 0.         -0.04567066]
 [ 0.04520223  0.        ]
 [ 0.04520223  0.04567066]
 [ 0.04520223 -0.04567066]
 [-0.04520223  0.        ]
 [-0.04520223  0.04567066]
 [-0.04520223 -0.04567066]]


The second method involves the use of the function
``prismatique.sample.S_matrix_k_xy_vectors``.

In [35]:
probe_model_params = stem_system_model_params.core_attrs["probe_model_params"]

kwargs = {"sample_specification": sample_specification,
          "probe_model_params": probe_model_params}
S_matrix_k_xy_vectors = prismatique.sample.S_matrix_k_xy_vectors(**kwargs)

msg = "S-matrix k_xy vectors (in 1/Å):"
print(msg)
print(np.array(S_matrix_k_xy_vectors))

S-matrix k_xy vectors (in 1/Å):
[[ 0.          0.        ]
 [ 0.          0.04567066]
 [ 0.         -0.04567066]
 [ 0.04520223  0.        ]
 [ 0.04520223  0.04567066]
 [ 0.04520223 -0.04567066]
 [-0.04520223  0.        ]
 [-0.04520223  0.04567066]
 [-0.04520223 -0.04567066]]


### Extract the data size of each subset of $S$-matrices that one could generate according the sample model ###

In [36]:
func_alias = prismatique.sample.S_matrix_subset_data_size

for subset_idx in range(num_frozen_phonon_config_subsets):
    kwargs = {"sample_specification": sample_specification,
              "probe_model_params": probe_model_params,
              "subset_idx": subset_idx}
    S_matrix_subset_data_size = func_alias(sample_specification,
                                           probe_model_params,
                                           subset_idx)
    
    unformatted_msg = "data size of S-matrix subset {} (in bytes): {}"
    msg = unformatted_msg.format(subset_idx, S_matrix_subset_data_size)
    print(msg)

data size of S-matrix subset 0 (in bytes): 1474560
data size of S-matrix subset 1 (in bytes): 1474560


### Extract the data size of a set of $S$-matrices that one could generate according the sample model ###

In [37]:
kwargs = {"sample_specification": sample_specification,
          "probe_model_params": probe_model_params}
S_matrix_set_data_size = prismatique.sample.S_matrix_set_data_size(**kwargs)

unformatted_msg = "data size of S-matrix set (in bytes): {}"
msg = unformatted_msg.format(S_matrix_set_data_size)
print(msg)

data size of S-matrix set (in bytes): 2949120


## Extracting properties that are only well-defined for the HRTEM simulation parameter set ##

### Extract the beam tilt step size ###

In [38]:
gun_model_params = hrtem_system_model_params.core_attrs["gun_model_params"]
mean_beam_energy = gun_model_params.core_attrs["mean_beam_energy"]

beam_tilt_step_size = prismatique.tilt.step_size(sample_specification,
                                                 mean_beam_energy)

unformatted_msg = "beam tilt step size (in mrads): {}"
msg = unformatted_msg.format(beam_tilt_step_size)
print(msg)

beam tilt step size (in mrads): (3.882198896562731, 3.922430011461781)


### Extract the beam tilts ###

This can be done in two ways. The first method involves the use of the function
``prismatique.load.hrtem_beam_tilts``:

In [39]:
beam_tilts = prismatique.load.hrtem_beam_tilts(path_to_hrtem_sim_params)

msg = "beam tilts (in mrads):"
print(msg)
print(np.array(beam_tilts))

beam tilts (in mrads):
[[-7.76439779 -3.92243001]
 [-7.76439779  0.        ]
 [-7.76439779  3.92243001]
 [-3.8821989  -7.84486002]
 [-3.8821989  -3.92243001]
 [-3.8821989   0.        ]
 [-3.8821989   3.92243001]
 [-3.8821989   7.84486002]
 [ 0.         -7.84486002]
 [ 0.         -3.92243001]
 [ 0.          0.        ]
 [ 0.          3.92243001]
 [ 0.          7.84486002]
 [ 3.8821989  -7.84486002]
 [ 3.8821989  -3.92243001]
 [ 3.8821989   0.        ]
 [ 3.8821989   3.92243001]
 [ 3.8821989   7.84486002]
 [ 7.76439779 -3.92243001]
 [ 7.76439779  0.        ]
 [ 7.76439779  3.92243001]]


The second method involves the use of the function ``prismatique.tilt.series``.

In [40]:
tilt_params = hrtem_system_model_params.core_attrs["tilt_params"]

beam_tilt_series = prismatique.tilt.series(sample_specification,
                                           mean_beam_energy, 
                                           tilt_params)

msg = "beam tilt series (in mrads):"
print(msg)
print(np.array(beam_tilt_series))

beam tilt series (in mrads):
[[-7.76439779 -3.92243001]
 [-7.76439779  0.        ]
 [-7.76439779  3.92243001]
 [-3.8821989  -7.84486002]
 [-3.8821989  -3.92243001]
 [-3.8821989   0.        ]
 [-3.8821989   3.92243001]
 [-3.8821989   7.84486002]
 [ 0.         -7.84486002]
 [ 0.         -3.92243001]
 [ 0.          0.        ]
 [ 0.          3.92243001]
 [ 0.          7.84486002]
 [ 3.8821989  -7.84486002]
 [ 3.8821989  -3.92243001]
 [ 3.8821989   0.        ]
 [ 3.8821989   3.92243001]
 [ 3.8821989   7.84486002]
 [ 7.76439779 -3.92243001]
 [ 7.76439779  0.        ]
 [ 7.76439779  3.92243001]]


### Extract the data size of the HRTEM simulation output dataset that one could generate according to the HRTEM simulation parameter set ###

In [41]:
output_params = hrtem_sim_params.core_attrs["output_params"]
kwargs = {"hrtem_system_model_params": hrtem_system_model_params,
          "output_params": output_params}
hrtem_output_data_size = prismatique.hrtem.output.data_size(**kwargs)

unformatted_msg = "data size of HRTEM simulation output (in bytes): {}"
msg = unformatted_msg.format(hrtem_output_data_size)
print(msg)

data size of HRTEM simulation output (in bytes): 20647936
